In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlite3

# Read in recipes

In [39]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

# Create table
c.execute('''select a.recipe_name,
                    c.ingredient,
                    a.quantity,
                    c.unit_measure
                    
             from weekly_planner_recipe_fact a
             
             left join weekly_planner_recipe b 
             on a.recipe_name = b.recipe_name
             
             left join weekly_planner_ingredients c
             on a.ingredient_id = c.ingredient_id

             ''')

cols = [x[0] for x in c.description]
df = pd.DataFrame(c.fetchall(), columns = cols)

c.close()

In [52]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

# Create table
c.execute('''select *
                    
             from weekly_planner_ingredients


             ''')

cols = [x[0] for x in c.description]
ingr = pd.DataFrame(c.fetchall(), columns = cols)

c.close()

In [54]:
ingr[ingr['ingredient'].str.contains('Acciughe')]

,ingredient,timestamp,ingredient_id,unit_measure
576,Acciughe sotto sale,2020-10-18 18:04:49.247357,Acciughe sotto sale_g,g
577,Acciughe sotto olio,2020-10-18 18:04:49.247357,Acciughe sotto olio_g,g


In [49]:
people = 7


ing_df = sub[['ingredient', 'quantity', 'unit_measure' ]].copy()

ing_df['quantity'] = ing_df['quantity'].astype(float).fillna(0)

# Group by ingredient and unit_measure
ing_df = ing_df.groupby(['ingredient', 'unit_measure'], as_index=False).sum()
ing_df = ing_df.sort_values(by = 'ingredient')
ing_df = ing_df.set_index('ingredient')

# Multiply quantities by number of people
ing_df['quantity'] = ing_df['quantity'] * people

# Correct quantity

new_quantities = []

for i in ing_df.itertuples():
    
    if i.unit_measure not in ['g', 'kg', 'ml', 'l']:
        new_quantities.append(i.quantity)
    else:
        new_quantity = i.quantity+50 - (i.quantity % 50)
        new_quantities.append(new_quantity)

ing_df['quantity'] = new_quantities

In [50]:
ing_df

,unit_measure,quantity
ingredient,,
Acqua,ml,2350.0
Aglio,spicchio,7.0
Bietole,g,400.0
Carote,g,100.0
Cavolo nero,g,400.0
Cipolle,g,100.0
Fagioli cannellini secchi,g,450.0
Olio extravergine di oliva,ml,50.0
Pane raffermo,g,300.0


In [10]:
class ShoppingList(object):

    def __init__(self, ingredient_tuples, people):

        self.ingredient_tuples = ingredient_tuples
        self.people = people

    def create_shopping_list(self):

        # Create Ingredients Pandas DataFrame
        vars = ['ingredient', 'quantity', 'unit_measure']
        ing_df = pd.DataFrame(self.ingredient_tuples, columns=vars)
        ing_df['quantity'] = ing_df['quantity'].astype(float).fillna(0)

        # Group by ingredient and unit_measure
        ing_df = ing_df.groupby(['ingredient', 'unit_measure'], as_index=False).sum()
        ing_df = ing_df.sort_values(by = 'ingredient')
        ing_df = ing_df.set_index('ingredient')

        # Multiply quantities by number of people
        ing_df['quantity'] = ing_df['quantity'] * self.people
        
        # Correct quantity by rounding to nearest 50-s
        new_quantities = []

        for i in ing_df.itertuples():

            if i.unit_measure not in ['g', 'kg', 'ml', 'l']:
                new_quantities.append(i.quantity)
            else:
                new_quantity = i.quantity+50 - (i.quantity % 50)
                new_quantities.append(new_quantity)
        ing_df['quantity'] = new_quantities

        # Convert to str and replace 0 values with empty string
        ing_df['quantity'] = ing_df['quantity'].astype('int').astype('str')
        ing_df['quantity'] = ing_df['quantity'].replace('0', ' ')

        # Replace nan in unite unit measure with empty string
        ing_df['unit_measure'] = ing_df['unit_measure'].astype('str')
        ing_df['unit_measure'] = ing_df['unit_measure'].replace('nan', ' ')

        # Create ingredients list
        shopping_list = ing_df.transpose().to_dict()

        return shopping_list



,id,quantity,timestamp,recipe_name,ingredient_id,recipe_name,recipe_source,url,timestamp,ingredient,timestamp,ingredient_id,unit_measure
0,11764,0,2020-10-18 12:42:11.658078,Pasta allo scarpariello,Peperoncino fresco_unit,Pasta allo scarpariello,GF,https://ricette.giallozafferano.it/Pasta-allo-...,2020-10-18 12:42:09.644515,Peperoncino fresco,2020-10-18 12:42:07.237903,Peperoncino fresco_unit,unit
1,11765,0,2020-10-18 12:42:11.658078,Paella vegetariana,Zafferano_bustina,Paella vegetariana,GF,https://ricette.giallozafferano.it/Paella-vege...,2020-10-18 12:42:09.644515,Zafferano,2020-10-18 12:42:07.237903,Zafferano_bustina,bustina
2,11766,0,2020-10-18 12:42:11.658078,Ciambella di riso ripiena,Zafferano_bustina,Ciambella di riso ripiena,GF,https://ricette.giallozafferano.it/Ciambella-d...,2020-10-18 12:42:09.644515,Zafferano,2020-10-18 12:42:07.237903,Zafferano_bustina,bustina
3,11767,0,2020-10-18 12:42:11.658078,Risotto alla salsiccia,Zafferano_bustina,Risotto alla salsiccia,GF,https://ricette.giallozafferano.it/Risotto-all...,2020-10-18 12:42:09.644515,Zafferano,2020-10-18 12:42:07.237903,Zafferano_bustina,bustina
4,11768,0,2020-10-18 12:42:11.658078,Cous cous all italiana,Zafferano_bustina,Cous cous all italiana,GF,https://ricette.giallozafferano.it/Cous-cous-a...,2020-10-18 12:42:09.644515,Zafferano,2020-10-18 12:42:07.237903,Zafferano_bustina,bustina


In [16]:
recipes = df['recipe_name']

In [17]:
recipes

,recipe_name,recipe_name
0,Pasta allo scarpariello,Pasta allo scarpariello
1,Paella vegetariana,Paella vegetariana
2,Ciambella di riso ripiena,Ciambella di riso ripiena
3,Risotto alla salsiccia,Risotto alla salsiccia
4,Cous cous all italiana,Cous cous all italiana
...,...,...
3648,Risotto alla salsiccia e nebbiolo,Risotto alla salsiccia e nebbiolo
3649,Spaghetti all acqua pazza,Spaghetti all acqua pazza
3650,Spaghetti all acqua pazza,Spaghetti all acqua pazza
3651,Lasagne vegetariane cupcake,Lasagne vegetariane cupcake
